### SettingWithCopyWarning

**Note**:
- This is a warning. Actual work might have been already been done despite the warning.
- This warning is usually encountered while performing chain indexing 
- This warning is encountered while modifying a view or a copy

_Let's check this with an example_

In [9]:
import pandas as pd
import numpy as np

# creating a sample dataframe
df = pd.DataFrame({
    'age' :     [ 20, 32, 23, 22, 32, 21, 37],
    'section' : [ 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII'],
    'city' :    [ 'Kolkata', 'Delhi', 'Mumbai', 'Kolkata', 'Mumbai', 'Delhi', 'Bangalore'],
    'gender' :  [ 'M', 'M', 'F', 'F', 'M', 'F', 'F'],
    'favourite_color' : [ np.NAN, np.NAN, 'black', np.NAN, 'white', 'red', 'orange']
},index=list("ABCDEFG"))

In [10]:
# view the data
df

,age,section,city,gender,favourite_color
A,20,I,Kolkata,M,NaN
B,32,II,Delhi,M,NaN
C,23,III,Mumbai,F,black
D,22,IV,Kolkata,F,NaN
E,32,V,Mumbai,M,white
F,21,VI,Delhi,F,red
G,37,VII,Bangalore,F,orange


#### Scenario 1:

Let's try to update the `favourite_color` for all `age > 30` to cyan 

In [11]:
# Listing out all rows for age > 30
df[df["age"] > 30]

,age,section,city,gender,favourite_color
B,32,II,Delhi,M,NaN
E,32,V,Mumbai,M,white
G,37,VII,Bangalore,F,orange


In [12]:
# Trying to set the color column by chaining operations (this should give out the warning)
df[df["age"] > 30]["favourite_color"] = "cyan"

/Users/nsbhatta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


- As observed above, we have got the warning
- Additionally, data never got updated in the original dataframe
- Reason for the warning: we tried updating a view 
> df[df["age"] > 30] generates a view from the original dataframe

In [13]:
df[df["age"] > 30]

,age,section,city,gender,favourite_color
B,32,II,Delhi,M,NaN
E,32,V,Mumbai,M,white
G,37,VII,Bangalore,F,orange
